<a href="https://colab.research.google.com/github/oraziotorre/MomentumShiftAI/blob/main/TennisDataGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Pandas per la gestione dei dati
import pandas as pd
import requests

from collections import Counter
from collections import defaultdict

# Per visualizzare i DataFrame su Google Colab in formato tabellare
from google.colab import data_table
data_table.enable_dataframe_formatter()

## Data Ingestion

Importo i csv grezzi dal MatchChartingProject su GitHub

In [ ]:
url = "https://raw.githubusercontent.com/JeffSackmann/tennis_MatchChartingProject/master/charting-m-points-2020s.csv"
response = requests.get(url)

with open("charting-m-points-2020s.csv", "w") as f:
    f.write(response.text)

dataset1 = pd.read_csv("charting-m-points-2020s.csv")


url = "https://raw.githubusercontent.com/JeffSackmann/tennis_MatchChartingProject/master/charting-m-points-2010s.csv"
response = requests.get(url)

with open("charting-m-points-2010s.csv", "w") as f:
    f.write(response.text)

dataset2 = pd.read_csv("charting-m-points-2010s.csv")


url = "https://raw.githubusercontent.com/JeffSackmann/tennis_MatchChartingProject/master/charting-m-points-to-2009.csv"
response = requests.get(url)

with open("charting-m-points-to-2009.csv", "w") as f:
    f.write(response.text)

dataset3 = pd.read_csv("charting-m-points-to-2009.csv")


dataset = pd.concat([dataset1, dataset2, dataset3], ignore_index=True)

dataset

/tmp/ipython-input-452344.py:7: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset1 = pd.read_csv("charting-m-points-2020s.csv")


,match_id,Pt,Set1,Set2,Gm1,Gm2,Pts,Gm#,TbSet,Svr,1st,2nd,Notes,PtWinner
0,20250610-M-ITF_Martos-Q2-Preston_Stearns-Aleja...,1,0,0,0.0,0.0,0-0,1.0,True,1,6#,NaN,NaN,1
1,20250610-M-ITF_Martos-Q2-Preston_Stearns-Aleja...,2,0,0,0.0,0.0,15-0,1.0,True,1,4x,5b2f3n@,NaN,2
2,20250610-M-ITF_Martos-Q2-Preston_Stearns-Aleja...,3,0,0,0.0,0.0,15-15,1.0,True,1,5d,5b2f1s3w#,NaN,1
3,20250610-M-ITF_Martos-Q2-Preston_Stearns-Aleja...,4,0,0,0.0,0.0,30-15,1.0,True,1,4f2f2d@,NaN,NaN,2
4,20250610-M-ITF_Martos-Q2-Preston_Stearns-Aleja...,5,0,0,0.0,0.0,30-30,1.0,True,1,4b2f3f2f2n@,NaN,NaN,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1126736,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,247,2,2,4.0,3.0,0-40,46.0,False,2,R,NaN,NaN,1
1126737,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,248,2,2,5.0,3.0,0-0,47.0,False,1,S,NaN,NaN,1
1126738,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,249,2,2,5.0,3.0,15-0,47.0,False,1,S,NaN,NaN,1
1126739,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,250,2,2,5.0,3.0,30-0,47.0,False,1,S,NaN,NaN,1


### Data Ingestion del ranking dei giocatori

In [ ]:
url = "https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_players.csv"
response = requests.get(url)

with open("atp_players.csv", "w") as f:
    f.write(response.text)

playerset = pd.read_csv("atp_players.csv")

# Sostituisci NaN con stringhe vuote prima della concatenazione
first = playerset['name_first'].fillna('').str.replace(' ', '_', regex=False).str.replace('-', '_', regex=False)
last = playerset['name_last'].fillna('').str.replace(' ', '_', regex=False).str.replace('-', '_', regex=False)

# Costruisci il full_name in modo sicuro
playerset['full_name'] = (
    first + '_' + last
).str.strip('_')  # Rimuove eventuali underscore iniziali/finali se una parte è vuota
playerset['full_name'] = playerset['full_name'].str.lower()
playerset

/tmp/ipython-input-990322619.py:7: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  playerset = pd.read_csv("atp_players.csv")


,player_id,name_first,name_last,hand,dob,ioc,height,wikidata_id,full_name
0,100001,Gardnar,Mulloy,R,19131122.0,USA,185.0,Q54544,gardnar_mulloy
1,100002,Pancho,Segura,R,19210620.0,ECU,168.0,Q54581,pancho_segura
2,100003,Frank,Sedgman,R,19271002.0,AUS,180.0,Q962049,frank_sedgman
3,100004,Giuseppe,Merlo,R,19271011.0,ITA,NaN,Q1258752,giuseppe_merlo
4,100005,Richard,Gonzalez,R,19280509.0,USA,188.0,Q53554,richard_gonzalez
...,...,...,...,...,...,...,...,...,...
65984,213700,Matvei,Kobiakov,U,NaN,RUS,NaN,NaN,matvei_kobiakov
65985,213701,Tobia Costanzo,Baragiola Mordini,U,NaN,ITA,NaN,NaN,tobia_costanzo_baragiola_mordini
65986,213702,Dominik,Wijntjes,U,NaN,NZL,NaN,NaN,dominik_wijntjes
65987,213703,Sam,Wensley,U,NaN,AUS,NaN,NaN,sam_wensley


Prendo da GitHub i rankings degli ultimi 20 anni dei giocatori di tennis

In [ ]:
url = "https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_rankings_current.csv"
response = requests.get(url)

with open("atp_rankings_current.csv", "w") as f:
    f.write(response.text)

ranking1 = pd.read_csv("atp_rankings_current.csv")


url = "https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_rankings_20s.csv"
response = requests.get(url)

with open("atp_rankings_20s.csv", "w") as f:
    f.write(response.text)

ranking2 = pd.read_csv("atp_rankings_20s.csv")


url = "https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_rankings_10s.csv"
response = requests.get(url)

with open("atp_rankings_10s.csv", "w") as f:
    f.write(response.text)

ranking3 = pd.read_csv("atp_rankings_10s.csv")


url = "https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_rankings_00s.csv"
response = requests.get(url)

with open("atp_rankings_00s.csv", "w") as f:
    f.write(response.text)

ranking4 = pd.read_csv("atp_rankings_00s.csv")


url = "https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_rankings_90s.csv"
response = requests.get(url)

with open("atp_rankings_90s.csv", "w") as f:
    f.write(response.text)

ranking5 = pd.read_csv("atp_rankings_90s.csv")


url = "https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_rankings_80s.csv"
response = requests.get(url)

with open("atp_rankings_80s.csv", "w") as f:
    f.write(response.text)

ranking6 = pd.read_csv("atp_rankings_80s.csv")


url = "https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_rankings_70s.csv"
response = requests.get(url)

with open("atp_rankings_70s.csv", "w") as f:
    f.write(response.text)

ranking7 = pd.read_csv("atp_rankings_70s.csv")

rankingset = pd.concat([ranking1, ranking2, ranking3, ranking4, ranking5, ranking6, ranking7], ignore_index=True)

rankingset

,ranking_date,rank,player,points
0,20240101,1,104925,11245.0
1,20240101,2,207989,8855.0
2,20240101,3,106421,7600.0
3,20240101,4,206173,6490.0
4,20240101,5,126094,4805.0
...,...,...,...,...
3292944,19771212,95,100586,NaN
3292945,19771212,97,100363,NaN
3292946,19771212,98,100176,NaN
3292947,19771212,99,100233,NaN


Ottimizzazione e Correzione degli errori presenti nel dataset originale

In [ ]:
# Crea il dizionario: full_name -> lista di player_id per ottimizzare l'esecuzione
player_id_map = defaultdict(list)

for _, row in playerset.iterrows():
  player_id_map[row['full_name']].append(row['player_id'])
  if(row['full_name']=='franko_skugor'):
    player_id_map['franco_skugor'].append(row['player_id'])
  if(row['full_name']=='jo_wilfried_tsonga'):
    player_id_map['jo_wilfred_tsonga'].append(row['player_id'])
  if(row['full_name']=='sam_groth'):
    player_id_map['samuel_groth'].append(row['player_id'])
  if(row['full_name']=='andres_artunedo_martinavarro'):
    player_id_map['andres_artunedo'].append(row['player_id'])
  if(row['full_name']=='marc_kevin_goellner'):
    player_id_map['mark_kevin_goellner'].append(row['player_id'])

rankingset['ranking_date'] = pd.to_datetime(rankingset['ranking_date'], format='%Y%m%d')

Creazione del dataset grezzo finale con i ranking dei giocatori

!!! Attenzione Tempo di Esecuzione minimo: 6 min

In [ ]:
last_match = None
match_date = None
player_id_1 = None
player_id_2 = None
ranking_1 = None
ranking_2 = None

def extract_rankings(row):
    global last_match, match_date, player_id_1, player_id_2, ranking_1, ranking_2
    match_id = row['match_id']
    if match_id == last_match:
      return pd.Series([match_date.strftime("%Y-%m-%d"), player_id_1, player_id_2, ranking_1, ranking_2])
    else:
      last_match = match_id

      match_info = match_id.split('-')
      ranking_date = match_info[0]
      name1 = match_info[-2].lower()
      name2 = match_info[-1].lower()

      # Trova i player_id in modo sicuro
      player_id1 = player_id_map.get(name1)
      player_id2 = player_id_map.get(name2)

      # Converte la data di riferimento 'ranking_date' in formato datetime
      match_date = pd.to_datetime(ranking_date, format='%Y%m%d')
      try:
        for player_id in player_id1:
            try:
                filtered_ranking = rankingset[
                    (rankingset['player'] == player_id) &
                    (rankingset['ranking_date'] <= match_date)
                ]
                if not filtered_ranking.empty:
                    ranking_1 = filtered_ranking.sort_values(by='ranking_date', ascending=False).iloc[0]['rank']
                    player_id_1 = player_id
                    break  # Esce dal ciclo appena trova un ranking valido
            except Exception as e:
                continue  # Salta in caso di errore e prova con il prossimo ID
      except Exception as e:
            print("Giocatore scartato")  # Salta in caso di errore e prova con il prossimo ID

      for player_id in player_id2:
          try:
              filtered_ranking = rankingset[
                  (rankingset['player'] == player_id) &
                  (rankingset['ranking_date'] <= match_date)
              ]
              if not filtered_ranking.empty:
                  ranking_2 = filtered_ranking.sort_values(by='ranking_date', ascending=False).iloc[0]['rank']
                  player_id_2 = player_id
                  break
          except Exception as e:
              continue

      return pd.Series([match_date.strftime("%Y-%m-%d"), player_id_1, player_id_2, ranking_1, ranking_2])


dataset[['Date', 'Player1', 'Player2', 'Ranking1', 'Ranking2']] = dataset.apply(extract_rankings, axis=1)
dataset

Giocatore scartato


,match_id,Pt,Set1,Set2,Gm1,Gm2,Pts,Gm#,TbSet,Svr,1st,2nd,Notes,PtWinner,Date,Player1,Player2,Ranking1,Ranking2
0,20250610-M-ITF_Martos-Q2-Preston_Stearns-Aleja...,1,0,0,0.0,0.0,0-0,1.0,True,1,6#,NaN,NaN,1,2025-06-10,210475,NaN,2065,NaN
1,20250610-M-ITF_Martos-Q2-Preston_Stearns-Aleja...,2,0,0,0.0,0.0,15-0,1.0,True,1,4x,5b2f3n@,NaN,2,2025-06-10,210475,NaN,2065,NaN
2,20250610-M-ITF_Martos-Q2-Preston_Stearns-Aleja...,3,0,0,0.0,0.0,15-15,1.0,True,1,5d,5b2f1s3w#,NaN,1,2025-06-10,210475,NaN,2065,NaN
3,20250610-M-ITF_Martos-Q2-Preston_Stearns-Aleja...,4,0,0,0.0,0.0,30-15,1.0,True,1,4f2f2d@,NaN,NaN,2,2025-06-10,210475,NaN,2065,NaN
4,20250610-M-ITF_Martos-Q2-Preston_Stearns-Aleja...,5,0,0,0.0,0.0,30-30,1.0,True,1,4b2f3f2f2n@,NaN,NaN,2,2025-06-10,210475,NaN,2065,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1126736,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,247,2,2,4.0,3.0,0-40,46.0,False,2,R,NaN,NaN,1,1960-05-29,100113,100087.0,11,7.0
1126737,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,248,2,2,5.0,3.0,0-0,47.0,False,1,S,NaN,NaN,1,1960-05-29,100113,100087.0,11,7.0
1126738,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,249,2,2,5.0,3.0,15-0,47.0,False,1,S,NaN,NaN,1,1960-05-29,100113,100087.0,11,7.0
1126739,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,250,2,2,5.0,3.0,30-0,47.0,False,1,S,NaN,NaN,1,1960-05-29,100113,100087.0,11,7.0


##Feature Engineering

### Controllo valori nulli e rari

In [214]:
print(dataset.isna().sum())

match_id          0
Pt                0
Set1              0
Set2              0
Gm1               1
Gm2               2
Pts               0
Gm#               1
TbSet             1
Svr               0
1st               0
2nd          702766
Notes       1023383
PtWinner          0
Date              0
Player1           0
Player2         134
Ranking1          0
Ranking2        134
dtype: int64


In [216]:
dataset['Gm1'] = dataset['Gm1'].fillna(0).astype(int)
dataset['Gm2'] = dataset['Gm2'].fillna(0).astype(int)
dataset['Gm1'] = dataset['Gm1'].fillna(0)
dataset['TbSet'] = dataset['TbSet'].fillna
dataset = dataset.dropna(subset=['Player1', 'Player2']).reset_index(drop=True)
dataset = dataset.drop(columns=['Gm#', 'Notes', 'TbSet'], errors='ignore')
print(dataset.isna().sum())

match_id         0
Pt               0
Set1             0
Set2             0
Gm1              0
Gm2              0
Pts              0
Svr              0
1st              0
2nd         702678
PtWinner         0
Date             0
Player1          0
Player2          0
Ranking1         0
Ranking2         0
dtype: int64


In [217]:
def check_dataset(df, column_ranges=None):
    print("🔍 Controllo del dataset...\n")

    # 1. Valori nulli
    nulls = df.isnull().sum()
    nulls = nulls[nulls > 0]
    if not nulls.empty:
        print("⚠️  Colonne con valori nulli:")
        print(nulls)
    else:
        print("✅ Nessun valore nullo trovato.\n")

    # 2. Duplicati
    duplicate_rows = df.duplicated().sum()
    if duplicate_rows > 0:
        print(f"⚠️  Righe duplicate: {duplicate_rows}")
    else:
        print("✅ Nessuna riga duplicata.\n")

    # 3. Tipi di dato
    print("📊 Tipi di dato per colonna:")
    print(df.dtypes)
    print()

    # 4. Valori non numerici in colonne numeriche
    for col in df.select_dtypes(include=['object']).columns:
        try:
            pd.to_numeric(df[col])
            print(f"⚠️  La colonna '{col}' contiene stringhe ma potrebbe essere numerica.")
        except:
            pass

    # 5. Cardinalità sospetta
    print("\n🔢 Cardinalità delle colonne:")
    for col in df.columns:
        n_unique = df[col].nunique()
        if n_unique > 100 and df[col].dtype == 'object':
            print(f"⚠️  '{col}' ha {n_unique} valori unici (potrebbe contenere ID o errori)")
        else:
            print(f"{col}: {n_unique} valori unici")

    # 6. Range controlli se specificato
    if column_ranges:
        print("\n📏 Controllo range valori:")
        for col, (low, high) in column_ranges.items():
            if col in df.columns:
                out_of_bounds = df[(df[col] < low) | (df[col] > high)]
                if not out_of_bounds.empty:
                    print(f"⚠️  Colonna '{col}' ha {len(out_of_bounds)} valori fuori dal range [{low}, {high}]")
                else:
                    print(f"✅ Colonna '{col}' OK nel range [{low}, {high}]")

    print("\n✅ Controllo completato.")


check_dataset(dataset)

🔍 Controllo del dataset...

⚠️  Colonne con valori nulli:
2nd    702678
dtype: int64
⚠️  Righe duplicate: 1662
📊 Tipi di dato per colonna:
match_id     object
Pt            int64
Set1          int64
Set2          int64
Gm1           int64
Gm2           int64
Pts          object
Svr           int64
1st          object
2nd          object
PtWinner      int64
Date         object
Player1       int64
Player2     float64
Ranking1      int64
Ranking2    float64
dtype: object


🔢 Cardinalità delle colonne:
⚠️  'match_id' ha 6604 valori unici (potrebbe contenere ID o errori)
Pt: 980 valori unici
Set1: 4 valori unici
Set2: 4 valori unici
Gm1: 69 valori unici
Gm2: 70 valori unici
⚠️  'Pts' ha 145 valori unici (potrebbe contenere ID o errori)
Svr: 2 valori unici
⚠️  '1st' ha 298299 valori unici (potrebbe contenere ID o errori)
⚠️  '2nd' ha 233963 valori unici (potrebbe contenere ID o errori)
PtWinner: 2 valori unici
⚠️  'Date' ha 3923 valori unici (potrebbe contenere ID o errori)
Player1: 677 valo

Noto che ci sono valori non coerenti nelle colonne dei Set e dei Game. Decido di rimuovere tali partite

In [218]:
# Trova i match_id da escludere
invalid_match_ids = dataset.loc[dataset['Gm1'] > 60, 'match_id'].unique()

# Conta quante partite verranno rimosse
print(f"Partite rimosse: {len(invalid_match_ids)}")

# Filtra via tutte le righe con quei match_id
dataset = dataset[~dataset['match_id'].isin(invalid_match_ids)]

Partite rimosse: 1


In [219]:
# Trova i match_id da escludere (Set1 o Set2 > 2)
invalid_match_ids = dataset.loc[
    (dataset['Set1'] > 2) | (dataset['Set2'] > 2),
    'match_id'
].unique()

# Conta quante partite verranno rimosse
print(f"Partite rimosse: {len(invalid_match_ids)}")

# Filtra via tutte le righe con quei match_id
dataset = dataset[~dataset['match_id'].isin(invalid_match_ids)]

Partite rimosse: 2


**Controllo la configurazione dei punteggi**

In [220]:
def normalize_pts_from_server(df):
    def flip_pts(pts):
        try:
            a, b = pts.split('-')
            return f"{b}-{a}"
        except:
            return pts  # se c'è un errore di parsing, restituisci il valore originale

    df['Pts'] = df.apply(
        lambda row: flip_pts(row['Pts']) if row['Svr'] == 2 else row['Pts'],
        axis=1
    )
    return df

dataset = normalize_pts_from_server(dataset)

In [221]:
conteggio_pts = dataset['Pts'].value_counts()

for simbolo, conteggio in conteggio_pts.items():
    print(f"'{simbolo}': {conteggio}")

'0-0': 180612
'15-0': 89217
'0-15': 87974
'15-15': 79164
'40-40': 74287
'30-15': 59251
'15-30': 58464
'30-30': 53427
'30-0': 49802
'0-30': 48224
'40-30': 43929
'30-40': 43903
'40-15': 43210
'15-40': 42355
'AD-40': 37376
'40-AD': 36822
'40-0': 30299
'0-40': 29157
'1-0': 1885
'1-1': 1848
'0-1': 1532
'1-2': 1366
'2-2': 1347
'2-1': 1242
'3-2': 1151
'3-3': 1117
'2-3': 1048
'4-3': 951
'3-4': 949
'4-4': 947
'5-4': 933
'5-5': 886
'3-1': 850
'2-0': 828
'6-6': 826
'6-5': 822
'4-2': 816
'1-3': 812
'4-5': 806
'5-3': 800
'2-4': 767
'5-6': 751
'0-2': 741
'6-4': 734
'3-5': 715
'4-6': 647
'4-1': 562
'5-2': 558
'6-3': 552
'2-5': 521
'3-6': 504
'7-7': 478
'7-6': 445
'1-4': 430
'0-3': 405
'3-0': 404
'6-7': 389
'6-2': 369
'2-6': 336
'5-1': 335
'1-5': 269
'8-8': 261
'7-8': 254
'8-7': 231
'4-0': 214
'0-4': 194
'6-1': 193
'1-6': 184
'9-9': 158
'8-9': 137
'9-8': 133
'5-0': 123
'0-5': 103
'10-10': 86
'10-9': 79
'9-10': 79
'0-6': 59
'6-0': 53
'11-11': 50
'11-10': 45
'10-11': 41
'12-12': 26
'12-11': 26
'11-12': 

In [222]:
def normalizza_vantaggi(p):
    esclusi = {"0", "15", "30", "40", "AD"}

    try:
        sx, dx = p.split('-')
    except ValueError:
        return p  # Formato non valido

    if sx in esclusi or dx in esclusi:
        return p  # È un punteggio classico del game, non tiebreak

    try:
        sx_int = int(sx)
        dx_int = int(dx)
    except ValueError:
        return p  # Uno dei due non è un numero (es. errore di dati)

    # Ora siamo sicuri che sono numeri da tiebreak
    if abs(sx_int - dx_int) <= 1 and sx_int>=6 and dx_int>=6 :
        if sx_int == dx_int:
            return '6-6'
        elif sx_int > dx_int:
            return 'AD-6'
        else:
            return '6-AD'
    else:
        return p

dataset['Pts'] = dataset['Pts'].apply(normalizza_vantaggi)
dataset

,match_id,Pt,Set1,Set2,Gm1,Gm2,Pts,Svr,1st,2nd,PtWinner,Date,Player1,Player2,Ranking1,Ranking2
0,20250609-M-Stuttgart-R32-Benjamin_Bonzi-Jiri_L...,1,0,0,0,0,0-0,1,4#,NaN,1,2025-06-09,126127,208103.0,75,28.0
1,20250609-M-Stuttgart-R32-Benjamin_Bonzi-Jiri_L...,2,0,0,0,0,15-0,1,4d,4b28b3b3b3b2n@,1,2025-06-09,126127,208103.0,75,28.0
2,20250609-M-Stuttgart-R32-Benjamin_Bonzi-Jiri_L...,3,0,0,0,0,30-0,1,6#,NaN,1,2025-06-09,126127,208103.0,75,28.0
3,20250609-M-Stuttgart-R32-Benjamin_Bonzi-Jiri_L...,4,0,0,0,0,40-0,1,4#,NaN,1,2025-06-09,126127,208103.0,75,28.0
4,20250609-M-Stuttgart-R32-Benjamin_Bonzi-Jiri_L...,5,0,0,1,0,0-0,2,4*,NaN,2,2025-06-09,126127,208103.0,75,28.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1126602,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,247,2,2,4,3,40-0,2,R,NaN,1,1960-05-29,100113,100087.0,11,7.0
1126603,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,248,2,2,5,3,0-0,1,S,NaN,1,1960-05-29,100113,100087.0,11,7.0
1126604,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,249,2,2,5,3,15-0,1,S,NaN,1,1960-05-29,100113,100087.0,11,7.0
1126605,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,250,2,2,5,3,30-0,1,S,NaN,1,1960-05-29,100113,100087.0,11,7.0


## Gestione game
GameID e GameWinner

In [223]:
# Split 'Pts'
dataset[['Pt1_raw', 'Pt2_raw']] = dataset['Pts'].astype(str).str.split('-', expand=True)

# Set numerici
dataset['Gm1'] = pd.to_numeric(dataset['Gm1'], errors='coerce')
dataset['Gm2'] = pd.to_numeric(dataset['Gm2'], errors='coerce')

# Mappa punteggi normali
normal_score_map = {'0': 0, '15': 1, '30': 2, '40': 3, 'AD': 4, '1': 1, '2': 2, '3': 3, '4': 4}
dataset['Pt1'] = dataset['Pt1_raw'].map(normal_score_map)
dataset['Pt2'] = dataset['Pt2_raw'].map(normal_score_map)

# Definisci IsDeuce (solo game normali ormai)
normal_deuce = (
    ((dataset['Pt1'] == 3) & (dataset['Pt2'] == 3)) |   # 40-40
    ((dataset['Pt1'] == 3) & (dataset['Pt2'] == 4)) |   # 40-ADV
    ((dataset['Pt1'] == 4) & (dataset['Pt2'] == 3))     # ADV-40
)

# Crea la colonna GameID
dataset['GameID'] = dataset['match_id'] + dataset['Set1'].astype(str) + dataset['Set2'].astype(str) + '_G' + dataset['Gm1'].astype(str) + '-' + dataset['Gm2'].astype(str)
# Rimuovi colonne temporanee
dataset.drop(columns=['Pt1_raw', 'Pt2_raw'], inplace=True)

dataset

,match_id,Pt,Set1,Set2,Gm1,Gm2,Pts,Svr,1st,2nd,PtWinner,Date,Player1,Player2,Ranking1,Ranking2,Pt1,Pt2,GameID
0,20250609-M-Stuttgart-R32-Benjamin_Bonzi-Jiri_L...,1,0,0,0,0,0-0,1,4#,NaN,1,2025-06-09,126127,208103.0,75,28.0,0.0,0.0,20250609-M-Stuttgart-R32-Benjamin_Bonzi-Jiri_L...
1,20250609-M-Stuttgart-R32-Benjamin_Bonzi-Jiri_L...,2,0,0,0,0,15-0,1,4d,4b28b3b3b3b2n@,1,2025-06-09,126127,208103.0,75,28.0,1.0,0.0,20250609-M-Stuttgart-R32-Benjamin_Bonzi-Jiri_L...
2,20250609-M-Stuttgart-R32-Benjamin_Bonzi-Jiri_L...,3,0,0,0,0,30-0,1,6#,NaN,1,2025-06-09,126127,208103.0,75,28.0,2.0,0.0,20250609-M-Stuttgart-R32-Benjamin_Bonzi-Jiri_L...
3,20250609-M-Stuttgart-R32-Benjamin_Bonzi-Jiri_L...,4,0,0,0,0,40-0,1,4#,NaN,1,2025-06-09,126127,208103.0,75,28.0,3.0,0.0,20250609-M-Stuttgart-R32-Benjamin_Bonzi-Jiri_L...
4,20250609-M-Stuttgart-R32-Benjamin_Bonzi-Jiri_L...,5,0,0,1,0,0-0,2,4*,NaN,2,2025-06-09,126127,208103.0,75,28.0,0.0,0.0,20250609-M-Stuttgart-R32-Benjamin_Bonzi-Jiri_L...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1126602,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,247,2,2,4,3,40-0,2,R,NaN,1,1960-05-29,100113,100087.0,11,7.0,3.0,0.0,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...
1126603,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,248,2,2,5,3,0-0,1,S,NaN,1,1960-05-29,100113,100087.0,11,7.0,0.0,0.0,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...
1126604,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,249,2,2,5,3,15-0,1,S,NaN,1,1960-05-29,100113,100087.0,11,7.0,1.0,0.0,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...
1126605,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,250,2,2,5,3,30-0,1,S,NaN,1,1960-05-29,100113,100087.0,11,7.0,2.0,0.0,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...


Verifica incoerenza Pt

In [224]:
df = dataset.copy()

# pt precedente dentro lo stesso GameID
df['pt_prev'] = df.groupby('GameID')['Pt'].shift(1)

# violazione: pt non è esattamente pt_prev + 1 (ignora la prima riga del gruppo)
viol = df['pt_prev'].notna() & (df['Pt'] != df['pt_prev'] + 1)

# GameID problematici
bad_ids = df.loc[viol, 'GameID'].unique()

# 1) quante entità GameID hanno avuto problemi
n_bad_ids = len(bad_ids)
print(f"Numero di GameID con problemi: {n_bad_ids}")

dataset = dataset[~dataset['GameID'].isin(bad_ids)].copy()

Numero di GameID con problemi: 1618


Verifica incoerenza Pt1

In [225]:
df = dataset.copy()

# Ordina per sequenza punti (sostituisci 'Pt' con la tua colonna indice dei punti)
df = df.sort_values(['GameID', 'Pt'])

# Pt2 precedente nello stesso GameID
df['pt_prev'] = df.groupby('GameID')['Pt1'].shift(1)

# Violazioni: diminuisce oppure salta > 1
viol = df['pt_prev'].notna() & (
    (df['Pt1'] < df['pt_prev']) | (df['Pt1'] > df['pt_prev'] + 1)
)

bad_ids = df.loc[viol, 'GameID'].unique()
print(f"Numero di GameID con problemi: {len(bad_ids)}")
dataset = dataset[~dataset['GameID'].isin(bad_ids)].copy()

Numero di GameID con problemi: 11690


Verifica incoerenza Pt2

In [226]:
df = dataset.copy()

# Ordina per sequenza punti (sostituisci 'Pt' con la tua colonna indice dei punti)
df = df.sort_values(['GameID', 'Pt'])

# Pt2 precedente nello stesso GameID
df['pt_prev'] = df.groupby('GameID')['Pt2'].shift(1)

# Violazioni: diminuisce oppure salta > 1
viol = df['pt_prev'].notna() & (
    (df['Pt2'] < df['pt_prev']) | (df['Pt2'] > df['pt_prev'] + 1)
)

bad_ids = df.loc[viol, 'GameID'].unique()
print(f"Numero di GameID con problemi: {len(bad_ids)}")
dataset = dataset[~dataset['GameID'].isin(bad_ids)].copy()

Numero di GameID con problemi: 6351


Verifica incoerenza PtWinner con Gm1 e Gm2

In [227]:
df = dataset.groupby('GameID').tail(1)

# Calcolo g1_next e g2_next in base a PtWinner
g1_next = df['Gm1'] + (df['PtWinner'] == 1).astype(int)
g2_next = df['Gm2'] + (df['PtWinner'] == 2).astype(int)

# GameID atteso
expected_next_id = (
    df['match_id'].astype(str)
    + df['Set1'].astype(str)
    + df['Set2'].astype(str)
    + '_G'
    + g1_next.astype(int).astype(str)
    + '-'
    + g2_next.astype(int).astype(str)
)

# Condizione per saltare il check
skip_check = ((g1_next - g2_next).abs() >= 2) | ((g1_next + g2_next) >= 12)

# Controllo esistenza
all_ids = df['GameID'].astype(str)
exists = expected_next_id.isin(all_ids)

# GameID mancanti (non esistono e non sono da saltare)
missing_ids = df.loc[~exists & ~skip_check, 'GameID'].unique()

# 1) Conteggio
print(f"Numero di Game incorretti trovati: {len(missing_ids)}")

dataset = dataset[~dataset['GameID'].isin(missing_ids)].copy()

Numero di Game incorretti trovati: 10523


Generazione etichette dei Game

In [228]:
# Calcola GameWinner come il PtWinner dell'ultima riga per ogni GameID
dataset['GameWinner'] = dataset.groupby('GameID')['PtWinner'].transform('last')

OPZIONALE: Rimozione dei game di tiebreak, inutili per i successivi sviluppi

In [229]:
# Maschera tiebreak SOLO se 6-6
is_tiebreak = (dataset['Gm1'] == 6) & (dataset['Gm2'] == 6)

gameid_rimossi = dataset.loc[is_tiebreak, 'GameID'].nunique()
print(f"Tiebreak rimossi: {gameid_rimossi}")

# Rimuovi le righe in cui è tiebreak
dataset = dataset[~is_tiebreak].copy()

Tiebreak rimossi: 3342


Verifica coerenza Pt1 e Pt2 con Game Winner

In [230]:
# --- Ultime istanze per chiave ---
ultime_game  = dataset.groupby('GameID').tail(1)   # per filtro Pt1/Pt2/GameWinner

# --- FILTRO 1: Pt1, Pt2, GameWinner su ultime istanze per GameID ---
filtrati_pt = ultime_game[(
    ((ultime_game['Pt1'] == 3) & (ultime_game['Pt2'] <= 2) & (ultime_game['GameWinner'] == 2)) |
    ((ultime_game['Pt1'] <= 2) & (ultime_game['Pt2'] == 3) & (ultime_game['GameWinner'] == 1)) |
    ((ultime_game['Pt1'] == 4) & (ultime_game['Pt2'] == 3) & (ultime_game['GameWinner'] == 2)) |
    ((ultime_game['Pt1'] == 3) & (ultime_game['Pt2'] == 4) & (ultime_game['GameWinner'] == 1)) |
    ((ultime_game['Pt1'].isna()) | (ultime_game['Pt2'].isna()) )
)]

print(f"Game rimossi: {filtrati_pt.shape[0]}")

# Rimuovi tutte le righe di dataset che hanno GameID in filtrati_pt
dataset = dataset[~dataset['GameID'].isin(filtrati_pt['GameID'])]

Game rimossi: 1


#### Analisi Bilanciamento GameWinner

In [231]:
simboli_finali = dataset['GameWinner'].value_counts()

simboli_finali

,count
GameWinner,
1,407637
2,394730


### Numero di partite e numero di game del dataset

In [232]:
dataset = dataset.rename(columns={'match_id': 'MatchID'})
simboli_finali = dataset['MatchID'].nunique()
simboli_finali

6590

In [233]:
simboli_finali = dataset['GameID'].nunique()
simboli_finali

146744

## Tipologie di esiti finali
Questi valori sono stati compararti con rispettivo significato presente sul MatchChartingProject

In [234]:
simboli_finali = Counter()


for index,row in dataset.iterrows():
  if(pd.isna(row['2nd'])):
    ultimo_carattere = row['1st'][-1]
    if(ultimo_carattere == ";"):
      ultimo_carattere = row['1st'][-2]
    simboli_finali[ultimo_carattere] += 1

  else:
    ultimo_carattere = row['2nd'][-1]
    if(ultimo_carattere == ";"):
      ultimo_carattere = row['2nd'][-2]
    simboli_finali[ultimo_carattere] += 1




for simbolo, conteggio in simboli_finali.most_common():
    print(f"'{simbolo}': {conteggio}")

'#': 268236
'*': 256679
'@': 240606
'n': 11514
'd': 11308
'S': 7659
'w': 3079
'R': 2113
'x': 663
'C': 209
'g': 64
'Q': 54
'!': 29
' ': 27
'P': 25
'3': 24
'e': 21
'1': 15
'9': 8
'2': 6
'8': 6
'+': 4
'7': 3
'^': 2
'b': 2
'=': 2
'4': 1
')': 1
'v': 1
'p': 1
'c': 1
'.': 1
'o': 1
'm': 1
'z': 1


#### Correzione dei casi rari e enfasi a informazioni importanti in un formato più gestibile

La descrizione in formato semplice, una riga per ogni stringa:


- `1` = `noinfo_1`: il giocatore 1 vince il punto, ma non è chiaro se è ace, errore, o altro.

- `2` = `noinfo_2`: il giocatore 2 vince il punto, ma non è chiaro se è ace, errore, o altro.

- `3` = `serve_ace_1`: punto diretto con il servizio (ace) fatto dal giocatore 1.

- `4` = `serve_ace_2`: punto diretto con il servizio (ace) fatto dal giocatore 2.

- `5` = `serve_miss2_1`: il giocatore 1 vince il punto su un errore di servizio del giocatore 2.

- `6` = `serve_miss1_2`: il giocatore 2 vince il punto su un errore di servizio del giocatore 1.

- `7` = `rally_winner_1`: il giocatore 1 vince il punto con un colpo vincente durante lo scambio.

- `8` = `rally_winner_2`: il giocatore 2 vince il punto con un colpo vincente durante lo scambio.

- `9` = `rally_forced2_1`: il giocatore 1 forza un errore del giocatore 2 durante lo scambio.

- `A` = `rally_forced1_2`: il giocatore 2 forza un errore del giocatore 1 durante lo scambio.

- `B` = `rally_unforced1_2`: il giocatore 2 vince per un errore non forzato del giocatore 1.

- `C` = `rally_unforced2_1`: il giocatore 1 vince per un errore non forzato del giocatore 2.

Attualmente non ho aggiunto considerazioni sul primo e secondo servizio, e su posizionamenti della palla e tipologia di colpi giocati.

Ho considerato le challenges Hawk-Eye perse come errori forzati.
Errori forzati e winners sono stati considerati come la stessa cosa nei servizi, si potrebbe fare la stessa considerazione anche nei rally

In [235]:
for index, row in dataset.iterrows():
    winner = row["PtWinner"]

    #On first serve points
    if pd.isna(row['2nd']):
        numeri = [c for c in row['1st'] if c.isdigit()]
        ultimo = row['1st'][-1]
        if(ultimo_carattere == ";"):
          ultimo_carattere = row['1st'][-2]

        #On serve points
        if len(numeri) == 1:
            if (ultimo == '#' or ultimo == '*' or ultimo == 'C') and winner == 1:
                dataset.at[index, 'PointType'] = '3'
            elif (ultimo == '#' or ultimo == '*' or ultimo == 'C') and winner == 2:
                dataset.at[index, 'PointType'] = '4'
            elif (ultimo == '@' or ultimo == 'd' or ultimo == 'w' or ultimo == 'n' or ultimo == 'x'or ultimo == 'g') and winner == 1:
                dataset.at[index, 'PointType'] = '5'
            elif (ultimo == '@' or ultimo == 'd' or ultimo == 'w' or ultimo == 'n' or ultimo == 'x'or ultimo == 'g') and winner == 2:
                dataset.at[index, 'PointType'] = '6'
            else:
                if winner == 1:
                    dataset.at[index, 'PointType'] = '1'
                else:
                  dataset.at[index, 'PointType'] = '2'

        #On rally points
        else:
            if ultimo == '*' and winner == 1:
                dataset.at[index, 'PointType'] = '7'
            elif ultimo == '*' and winner == 2:
                dataset.at[index, 'PointType'] = '8'
            elif (ultimo == '#' or ultimo =="C") and winner == 1:
                dataset.at[index, 'PointType'] = '9'
            elif (ultimo == '#' or ultimo =="C") and winner == 2:
                dataset.at[index, 'PointType'] = 'A'
            elif ultimo == '@' and winner == 1:
                dataset.at[index, 'PointType'] = 'B'
            elif ultimo == '@' and winner == 2:
                dataset.at[index, 'PointType'] = 'C'
            else:
                if winner == 1:
                    dataset.at[index, 'PointType'] = '1'
                elif winner == 2:
                  dataset.at[index, 'PointType'] = '2'

    #On second serve points
    else:
        numeri = [c for c in row['2nd'] if c.isdigit()]
        ultimo = row['2nd'][-1]
        if(ultimo_carattere == ";"):
          ultimo_carattere = row['2nd'][-2]

        #On serve points
        if len(numeri) == 1:
            if (ultimo == '#' or ultimo == '*' or ultimo == 'C') and winner == 1:
                dataset.at[index, 'PointType'] = '3'
            elif (ultimo == '#' or ultimo == '*' or ultimo == 'C') and winner == 2:
                dataset.at[index, 'PointType'] = '4'
            elif (ultimo == '@' or ultimo == 'd' or ultimo == 'w' or ultimo == 'n' or ultimo == 'x'or ultimo == 'g') and winner == 1:
                dataset.at[index, 'PointType'] = '5'
            elif (ultimo == '@' or ultimo == 'd' or ultimo == 'w' or ultimo == 'n' or ultimo == 'x'or ultimo == 'g') and winner == 2:
                dataset.at[index, 'PointType'] = '6'
            else:
                if winner == 1:
                    dataset.at[index, 'PointType'] = '1'
                else:
                  dataset.at[index, 'PointType'] = '2'

        #On rally points
        else:
            if ultimo == '*' and winner == 1:
                dataset.at[index, 'PointType'] = '7'
            elif ultimo == '*' and winner == 2:
                dataset.at[index, 'PointType'] = '8'
            elif (ultimo == '#' or ultimo =="C") and winner == 1:
                dataset.at[index, 'PointType'] = '9'
            elif (ultimo == '#' or ultimo =="C") and winner == 2:
                dataset.at[index, 'PointType'] = 'A'
            elif ultimo == '@' and winner == 1:
                dataset.at[index, 'PointType'] = 'B'
            elif ultimo == '@' and winner == 2:
                dataset.at[index, 'PointType'] = 'C'
            else:
                if winner == 1:
                    dataset.at[index, 'PointType'] = '1'
                elif winner == 2:
                  dataset.at[index, 'PointType'] = '2'


dataset

,MatchID,Pt,Set1,Set2,Gm1,Gm2,Pts,Svr,1st,2nd,...,Date,Player1,Player2,Ranking1,Ranking2,Pt1,Pt2,GameID,GameWinner,PointType
0,20250609-M-Stuttgart-R32-Benjamin_Bonzi-Jiri_L...,1,0,0,0,0,0-0,1,4#,NaN,...,2025-06-09,126127,208103.0,75,28.0,0.0,0.0,20250609-M-Stuttgart-R32-Benjamin_Bonzi-Jiri_L...,1,3
1,20250609-M-Stuttgart-R32-Benjamin_Bonzi-Jiri_L...,2,0,0,0,0,15-0,1,4d,4b28b3b3b3b2n@,...,2025-06-09,126127,208103.0,75,28.0,1.0,0.0,20250609-M-Stuttgart-R32-Benjamin_Bonzi-Jiri_L...,1,B
2,20250609-M-Stuttgart-R32-Benjamin_Bonzi-Jiri_L...,3,0,0,0,0,30-0,1,6#,NaN,...,2025-06-09,126127,208103.0,75,28.0,2.0,0.0,20250609-M-Stuttgart-R32-Benjamin_Bonzi-Jiri_L...,1,3
3,20250609-M-Stuttgart-R32-Benjamin_Bonzi-Jiri_L...,4,0,0,0,0,40-0,1,4#,NaN,...,2025-06-09,126127,208103.0,75,28.0,3.0,0.0,20250609-M-Stuttgart-R32-Benjamin_Bonzi-Jiri_L...,1,3
4,20250609-M-Stuttgart-R32-Benjamin_Bonzi-Jiri_L...,5,0,0,1,0,0-0,2,4*,NaN,...,2025-06-09,126127,208103.0,75,28.0,0.0,0.0,20250609-M-Stuttgart-R32-Benjamin_Bonzi-Jiri_L...,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1126602,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,247,2,2,4,3,40-0,2,R,NaN,...,1960-05-29,100113,100087.0,11,7.0,3.0,0.0,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,1,1
1126603,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,248,2,2,5,3,0-0,1,S,NaN,...,1960-05-29,100113,100087.0,11,7.0,0.0,0.0,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,1,1
1126604,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,249,2,2,5,3,15-0,1,S,NaN,...,1960-05-29,100113,100087.0,11,7.0,1.0,0.0,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,1,1
1126605,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,250,2,2,5,3,30-0,1,S,NaN,...,1960-05-29,100113,100087.0,11,7.0,2.0,0.0,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,1,1


In [236]:
dataset = dataset.drop(columns=['1st', '2nd'], errors='ignore')

#### Controllo quantità tipologie di esiti finali

In [237]:
simboli_finali = dataset['PointType'].value_counts()

simboli_finali

,count
PointType,
C,120150
B,118083
9,117766
A,114217
7,93526
8,91876
3,55698
4,52041
6,14348


### Feature Derivate


- p1_win_nobreak_point: P1 vince un game point mentre serve (tiene il servizio)

- p1_win_break_point: P1 vince un break point mentre risponde

- p1_lost_nobreak_point: P1 perde un game point mentre serve

- p1_lost_break_point: P1 perde un break point mentre risponde

- p2_win_nobreak_point: P2 vince un game point mentre serve

- p2_win_break_point: P2 vince un break point mentre risponde

- p2_lost_nobreak_point: P2 perde un game point mentre serve

- p2_lost_break_point: P2 perde un break point mentre risponde


In [238]:
dataset['p1_win_nobreak_point'] = 0
dataset['p2_win_nobreak_point'] = 0
dataset['p1_win_break_point'] = 0
dataset['p2_win_break_point'] = 0
dataset['p1_lost_nobreak_point'] = 0
dataset['p2_lost_nobreak_point'] = 0
dataset['p1_lost_break_point'] = 0
dataset['p2_lost_break_point'] = 0

punteggi_6_x = [f'6-{i}' for i in range(6)]
punteggi_game_point = ['40-0', '40-15', '40-30', 'AD-40', 'AD-6'] + punteggi_6_x

punteggio_favorevole = dataset['Pts'].isin(punteggi_game_point)
p1_vittoria_punto = dataset['PtWinner'] == 1
servizio = dataset['Svr'] == 1

condizione = punteggio_favorevole & p1_vittoria_punto & servizio
dataset.loc[condizione, 'p1_win_nobreak_point'] = 1
condizione = punteggio_favorevole & p1_vittoria_punto & (~servizio)
dataset.loc[condizione, 'p1_win_break_point'] = 1
condizione = punteggio_favorevole & (~p1_vittoria_punto) & servizio
dataset.loc[condizione, 'p1_lost_nobreak_point'] = 1
condizione = punteggio_favorevole & (~p1_vittoria_punto) & (~servizio)
dataset.loc[condizione, 'p1_lost_break_point'] = 1


punteggi_6_x = [f'{i}-6' for i in range(6)]
punteggi_game_point = ['0-40', '15-40', '30-40', '40-AD', '6-AD'] + punteggi_6_x
punteggio_favorevole = dataset['Pts'].isin(punteggi_game_point)
p2_vittoria_punto = dataset['PtWinner'] == 2
servizio = dataset['Svr'] == 2

condizione = punteggio_favorevole & p2_vittoria_punto & servizio
dataset.loc[condizione, 'p2_win_nobreak_point'] = 1
condizione = punteggio_favorevole & p2_vittoria_punto & (~servizio)
dataset.loc[condizione, 'p2_win_break_point'] = 1
condizione = punteggio_favorevole & (~p2_vittoria_punto) & servizio
dataset.loc[condizione, 'p2_lost_nobreak_point'] = 1
condizione = punteggio_favorevole & (~p2_vittoria_punto) & (~servizio)
dataset.loc[condizione, 'p2_lost_break_point'] = 1

dataset

,MatchID,Pt,Set1,Set2,Gm1,Gm2,Pts,Svr,PtWinner,Date,...,GameWinner,PointType,p1_win_nobreak_point,p2_win_nobreak_point,p1_win_break_point,p2_win_break_point,p1_lost_nobreak_point,p2_lost_nobreak_point,p1_lost_break_point,p2_lost_break_point
0,20250609-M-Stuttgart-R32-Benjamin_Bonzi-Jiri_L...,1,0,0,0,0,0-0,1,1,2025-06-09,...,1,3,0,0,0,0,0,0,0,0
1,20250609-M-Stuttgart-R32-Benjamin_Bonzi-Jiri_L...,2,0,0,0,0,15-0,1,1,2025-06-09,...,1,B,0,0,0,0,0,0,0,0
2,20250609-M-Stuttgart-R32-Benjamin_Bonzi-Jiri_L...,3,0,0,0,0,30-0,1,1,2025-06-09,...,1,3,0,0,0,0,0,0,0,0
3,20250609-M-Stuttgart-R32-Benjamin_Bonzi-Jiri_L...,4,0,0,0,0,40-0,1,1,2025-06-09,...,1,3,1,0,0,0,0,0,0,0
4,20250609-M-Stuttgart-R32-Benjamin_Bonzi-Jiri_L...,5,0,0,1,0,0-0,2,2,2025-06-09,...,2,4,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1126602,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,247,2,2,4,3,40-0,2,1,1960-05-29,...,1,1,0,0,1,0,0,0,0,0
1126603,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,248,2,2,5,3,0-0,1,1,1960-05-29,...,1,1,0,0,0,0,0,0,0,0
1126604,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,249,2,2,5,3,15-0,1,1,1960-05-29,...,1,1,0,0,0,0,0,0,0,0
1126605,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,250,2,2,5,3,30-0,1,1,1960-05-29,...,1,1,0,0,0,0,0,0,0,0


Creazione delle feature derivate dalla precedente analisi del colpo con cui è finito lo scambio

In [239]:
mapping = {
    '3': 'serve_ace_1',
    '4': 'serve_ace_2',
    '5': 'serve_miss2_1',
    '6': 'serve_miss1_2',
    '7': 'rally_winner_1',
    '8': 'rally_winner_2',
    '9': 'rally_forced2_1',
    'A': 'rally_forced1_2',
    'B': 'rally_unforced1_2',
    'C': 'rally_unforced2_1'
}

# Per ogni simbolo nella mappa, crea una nuova colonna
for symbol, column in mapping.items():
    dataset[column] = (dataset['PointType'] == symbol).astype(int)

dataset

,MatchID,Pt,Set1,Set2,Gm1,Gm2,Pts,Svr,PtWinner,Date,...,serve_ace_1,serve_ace_2,serve_miss2_1,serve_miss1_2,rally_winner_1,rally_winner_2,rally_forced2_1,rally_forced1_2,rally_unforced1_2,rally_unforced2_1
0,20250609-M-Stuttgart-R32-Benjamin_Bonzi-Jiri_L...,1,0,0,0,0,0-0,1,1,2025-06-09,...,1,0,0,0,0,0,0,0,0,0
1,20250609-M-Stuttgart-R32-Benjamin_Bonzi-Jiri_L...,2,0,0,0,0,15-0,1,1,2025-06-09,...,0,0,0,0,0,0,0,0,1,0
2,20250609-M-Stuttgart-R32-Benjamin_Bonzi-Jiri_L...,3,0,0,0,0,30-0,1,1,2025-06-09,...,1,0,0,0,0,0,0,0,0,0
3,20250609-M-Stuttgart-R32-Benjamin_Bonzi-Jiri_L...,4,0,0,0,0,40-0,1,1,2025-06-09,...,1,0,0,0,0,0,0,0,0,0
4,20250609-M-Stuttgart-R32-Benjamin_Bonzi-Jiri_L...,5,0,0,1,0,0-0,2,2,2025-06-09,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1126602,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,247,2,2,4,3,40-0,2,1,1960-05-29,...,0,0,0,0,0,0,0,0,0,0
1126603,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,248,2,2,5,3,0-0,1,1,1960-05-29,...,0,0,0,0,0,0,0,0,0,0
1126604,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,249,2,2,5,3,15-0,1,1,1960-05-29,...,0,0,0,0,0,0,0,0,0,0
1126605,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,250,2,2,5,3,30-0,1,1,1960-05-29,...,0,0,0,0,0,0,0,0,0,0


In [240]:
dataset= dataset.drop(columns=['Date'])

# Riposizionamento delle nuove colonne
is_final_index = dataset.columns.get_loc('Svr')
player1_column = dataset.pop('Player1')
player2_column = dataset.pop('Player2')
ranking1_column = dataset.pop('Ranking1')
ranking2_column = dataset.pop('Ranking2')

dataset.insert(is_final_index, 'Player1', player1_column)
dataset.insert(is_final_index + 1, 'Player2', player2_column)
dataset.insert(is_final_index + 2, 'Ranking1', ranking1_column)
dataset.insert(is_final_index + 3, 'Ranking2', ranking2_column)


dataset

,MatchID,Pt,Set1,Set2,Gm1,Gm2,Pts,Player1,Player2,Ranking1,...,serve_ace_1,serve_ace_2,serve_miss2_1,serve_miss1_2,rally_winner_1,rally_winner_2,rally_forced2_1,rally_forced1_2,rally_unforced1_2,rally_unforced2_1
0,20250609-M-Stuttgart-R32-Benjamin_Bonzi-Jiri_L...,1,0,0,0,0,0-0,126127,208103.0,75,...,1,0,0,0,0,0,0,0,0,0
1,20250609-M-Stuttgart-R32-Benjamin_Bonzi-Jiri_L...,2,0,0,0,0,15-0,126127,208103.0,75,...,0,0,0,0,0,0,0,0,1,0
2,20250609-M-Stuttgart-R32-Benjamin_Bonzi-Jiri_L...,3,0,0,0,0,30-0,126127,208103.0,75,...,1,0,0,0,0,0,0,0,0,0
3,20250609-M-Stuttgart-R32-Benjamin_Bonzi-Jiri_L...,4,0,0,0,0,40-0,126127,208103.0,75,...,1,0,0,0,0,0,0,0,0,0
4,20250609-M-Stuttgart-R32-Benjamin_Bonzi-Jiri_L...,5,0,0,1,0,0-0,126127,208103.0,75,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1126602,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,247,2,2,4,3,40-0,100113,100087.0,11,...,0,0,0,0,0,0,0,0,0,0
1126603,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,248,2,2,5,3,0-0,100113,100087.0,11,...,0,0,0,0,0,0,0,0,0,0
1126604,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,249,2,2,5,3,15-0,100113,100087.0,11,...,0,0,0,0,0,0,0,0,0,0
1126605,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,250,2,2,5,3,30-0,100113,100087.0,11,...,0,0,0,0,0,0,0,0,0,0


## Salvataggio dataset

In [241]:
# 1️⃣ Rimuovere la colonna 'PointType'
if "PointType" in dataset.columns:
    dataset = dataset.drop(columns=["PointType"])

# 2️⃣ Ottenere lista colonne attuali
cols = list(dataset.columns)

# Funzione per spostare una colonna dopo un'altra
def move_col(cols_list, col_to_move, col_after):
    if col_to_move in cols_list and col_after in cols_list:
        cols_list.remove(col_to_move)
        idx = cols_list.index(col_after) + 1
        cols_list.insert(idx, col_to_move)
    return cols_list

# 3️⃣ Spostare Pt1 e Pt2 dopo Gm2
cols = move_col(cols, "Pt1", "Gm2")
cols = move_col(cols, "Pt2", "Pt1")  # Pt2 subito dopo Pt1

# 4️⃣ Spostare GameID subito dopo MatchID
cols = move_col(cols, "GameID", "MatchID")

# 5️⃣ Spostare Svr subito dopo Pt
cols = move_col(cols, "Svr", "Pt")

# 6️⃣ Spostare GameWinner subito dopo PtWinner
cols = move_col(cols, "GameWinner", "PtWinner")

# 7️⃣ Riordinare il DataFrame
dataset = dataset[cols]

In [242]:
dataset.to_csv('tennis.csv')
dataset

,MatchID,GameID,Pt,Svr,Set1,Set2,Gm1,Gm2,Pt1,Pt2,...,serve_ace_1,serve_ace_2,serve_miss2_1,serve_miss1_2,rally_winner_1,rally_winner_2,rally_forced2_1,rally_forced1_2,rally_unforced1_2,rally_unforced2_1
0,20250609-M-Stuttgart-R32-Benjamin_Bonzi-Jiri_L...,20250609-M-Stuttgart-R32-Benjamin_Bonzi-Jiri_L...,1,1,0,0,0,0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0
1,20250609-M-Stuttgart-R32-Benjamin_Bonzi-Jiri_L...,20250609-M-Stuttgart-R32-Benjamin_Bonzi-Jiri_L...,2,1,0,0,0,0,1.0,0.0,...,0,0,0,0,0,0,0,0,1,0
2,20250609-M-Stuttgart-R32-Benjamin_Bonzi-Jiri_L...,20250609-M-Stuttgart-R32-Benjamin_Bonzi-Jiri_L...,3,1,0,0,0,0,2.0,0.0,...,1,0,0,0,0,0,0,0,0,0
3,20250609-M-Stuttgart-R32-Benjamin_Bonzi-Jiri_L...,20250609-M-Stuttgart-R32-Benjamin_Bonzi-Jiri_L...,4,1,0,0,0,0,3.0,0.0,...,1,0,0,0,0,0,0,0,0,0
4,20250609-M-Stuttgart-R32-Benjamin_Bonzi-Jiri_L...,20250609-M-Stuttgart-R32-Benjamin_Bonzi-Jiri_L...,5,2,0,0,1,0,0.0,0.0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1126602,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,247,2,2,2,4,3,3.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1126603,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,248,1,2,2,5,3,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1126604,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,249,1,2,2,5,3,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1126605,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,250,1,2,2,5,3,2.0,0.0,...,0,0,0,0,0,0,0,0,0,0
